# Developer Salary Prediction Model

This notebook builds a machine learning model to predict developer salaries based on Stack Overflow survey data.

## Table of Contents
1. [Data Loading and Exploration](#data-loading)
2. [Data Cleaning and Preprocessing](#data-cleaning)
3. [Exploratory Data Analysis](#eda)
4. [Feature Engineering](#feature-engineering)
5. [Model Training and Evaluation](#model-training)
6. [Model Optimization](#model-optimization)
7. [Final Model Persistence](#model-persistence)

## Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

## 1. Data Loading and Initial Exploration {#data-loading}

Loading the Stack Overflow Developer Survey data and performing initial exploration.
You can download data from here: https://survey.stackoverflow.co/

In [ ]:
# Load the dataset
df = pd.read_csv("data/survey_results_public.csv")

print(f"Dataset shape: {df.shape}")
print(f"\nColumns in dataset: {len(df.columns)}")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Display first few rows
df.head()

## 2. Data Cleaning and Preprocessing {#data-cleaning}

### 2.1 Feature Selection and Initial Cleaning

In [ ]:
# Select relevant columns for salary prediction
relevant_columns = [
    "Country", 
    "EdLevel", 
    "YearsCodePro", 
    "Employment", 
    "ConvertedCompYearly",
    "Age",
    "OrgSize",
    "DevType"
]

# Filter columns that exist in the dataset
available_columns = [col for col in relevant_columns if col in df.columns]
df_selected = df[available_columns].copy()

# Rename for clarity
df_selected = df_selected.rename({"ConvertedCompYearly": "Salary"}, axis=1)

print(f"Selected features: {list(df_selected.columns)}")
print(f"\nDataset shape after selection: {df_selected.shape}")

In [ ]:
# Data quality assessment
print("=== Data Quality Assessment ===")
print(f"\nMissing values per column:")
missing_data = df_selected.isnull().sum()
missing_percentage = (missing_data / len(df_selected)) * 100
missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percentage
})
print(missing_summary[missing_summary['Missing_Count'] > 0])

print(f"\nDuplicate rows: {df_selected.duplicated().sum()}")

### 2.2 Employment Status Filtering

In [ ]:
# Filter for valid salary data and full-time employment
print("Employment status distribution:")
print(df_selected['Employment'].value_counts())

# Keep only full-time employed individuals with salary data
df_clean = df_selected[
    (df_selected["Salary"].notnull()) & 
    (df_selected["Employment"] == "Employed, full-time")
].copy()

# Drop employment column as it's no longer needed
df_clean = df_clean.drop("Employment", axis=1)

print(f"\nRows after employment filtering: {len(df_clean)}")

### 2.3 Salary Data Cleaning and Outlier Detection

In [ ]:
# Salary distribution analysis
print("=== Salary Distribution Analysis ===")
print(df_clean['Salary'].describe())

# Visualize salary distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
axes[0].hist(df_clean['Salary'], bins=50, alpha=0.7, edgecolor='black')
axes[0].set_title('Salary Distribution')
axes[0].set_xlabel('Salary (USD)')
axes[0].set_ylabel('Frequency')

# Box plot
axes[1].boxplot(df_clean['Salary'])
axes[1].set_title('Salary Box Plot')
axes[1].set_ylabel('Salary (USD)')

# Log scale histogram
axes[2].hist(np.log10(df_clean['Salary']), bins=50, alpha=0.7, edgecolor='black')
axes[2].set_title('Log10 Salary Distribution')
axes[2].set_xlabel('Log10(Salary)')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Remove salary outliers using IQR method with domain knowledge
def remove_salary_outliers(df, column='Salary'):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Use broader range for salary data + domain knowledge
    lower_bound = max(Q1 - 2.5 * IQR, 10000)  # Minimum realistic salary
    upper_bound = min(Q3 + 2.5 * IQR, 500000)  # Maximum realistic salary
    
    print(f"Salary range: ${lower_bound:,.0f} - ${upper_bound:,.0f}")
    
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

df_clean = remove_salary_outliers(df_clean)
print(f"Rows after salary outlier removal: {len(df_clean)}")

### 2.4 Country Data Processing

In [ ]:
# Analyze country distribution
print("Top 20 countries by respondent count:")
country_counts = df_clean['Country'].value_counts()
print(country_counts.head(20))

# Visualize country distribution
plt.figure(figsize=(12, 8))
country_counts.head(15).plot(kind='bar')
plt.title('Top 15 Countries by Survey Respondents')
plt.xlabel('Country')
plt.ylabel('Number of Respondents')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
def shorten_categories(categories, cutoff):
    """Group low-frequency categories into 'Other' to reduce noise"""
    categorical_map = {}
    for category, count in categories.items():
        if count >= cutoff:
            categorical_map[category] = category
        else:
            categorical_map[category] = "Other"
    return categorical_map

# Apply country grouping with higher threshold for better model performance
country_map = shorten_categories(df_clean['Country'].value_counts(), 300)
df_clean['Country'] = df_clean['Country'].map(country_map)

print("Countries after grouping:")
print(df_clean['Country'].value_counts())

## 3. Exploratory Data Analysis {#eda}

### 3.1 Salary by Country Analysis

In [ ]:
# Remove 'Other' category for cleaner analysis
df_clean = df_clean[df_clean['Country'] != 'Other']

# Create enhanced salary by country visualization
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Box plot
df_clean.boxplot(column="Salary", by="Country", ax=axes[0])
axes[0].set_title("Salary Distribution by Country")
axes[0].set_ylabel("Salary (USD)")
axes[0].tick_params(axis='x', rotation=45)

# Mean salary by country
mean_salaries = df_clean.groupby('Country')['Salary'].mean().sort_values(ascending=False)
mean_salaries.plot(kind='bar', ax=axes[1], color='skyblue')
axes[1].set_title("Average Salary by Country")
axes[1].set_ylabel("Average Salary (USD)")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nAverage salaries by country:")
print(mean_salaries)

## 4. Feature Engineering {#feature-engineering}

### 4.1 Years of Experience Processing

In [ ]:
# Handle missing values first
print("Missing values before cleaning:")
print(df_clean.isnull().sum())

# Remove rows with missing critical features
df_clean = df_clean.dropna(subset=['Country', 'EdLevel', 'YearsCodePro'])

print(f"\nRows after removing missing values: {len(df_clean)}")
print("\nUnique values in YearsCodePro:")
print(df_clean['YearsCodePro'].unique())

In [ ]:
def clean_experience(x):
    """Convert experience categories to numeric values"""
    if pd.isna(x):
        return np.nan
    if "More than 50 years" in str(x):
        return 50
    if "Less than 1 year" in str(x):
        return 0.5
    try:
        return float(x)
    except:
        return np.nan

df_clean["YearsCodePro"] = df_clean["YearsCodePro"].apply(clean_experience)

# Remove any remaining NaN values in experience
df_clean = df_clean.dropna(subset=['YearsCodePro'])

print(f"Experience range: {df_clean['YearsCodePro'].min()} - {df_clean['YearsCodePro'].max()}")
print(f"\nExperience distribution:")
print(df_clean['YearsCodePro'].describe())

### 4.2 Education Level Processing

In [ ]:
print("Education level distribution:")
print(df_clean['EdLevel'].value_counts())

# Visualize education distribution
plt.figure(figsize=(10, 6))
df_clean['EdLevel'].value_counts().plot(kind='bar')
plt.title('Education Level Distribution')
plt.xlabel('Education Level')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Enhanced education mapping with better categorization
def clean_education(x):
    """Standardize education levels into meaningful categories"""
    if pd.isna(x):
        return "Unknown"
    
    x = str(x).lower()
    
    if "bachelor" in x:
        return "Bachelor's degree"
    elif "master" in x:
        return "Master's degree"
    elif "doctoral" in x or "phd" in x:
        return "Doctoral degree"
    elif "professional" in x or "law" in x or "medicine" in x:
        return "Professional degree"
    elif "associate" in x:
        return "Associate degree"
    elif "high school" in x or "secondary" in x:
        return "High school"
    else:
        return "Other/Some college"

df_clean["EdLevel"] = df_clean["EdLevel"].apply(clean_education)

print("Education levels after cleaning:")
print(df_clean["EdLevel"].value_counts())

### 4.3 Feature Encoding and Scaling

In [ ]:
# Create and store label encoders for later use
le_education = LabelEncoder()
le_country = LabelEncoder()

# Fit and transform categorical variables
df_encoded = df_clean.copy()
df_encoded['EdLevel_encoded'] = le_education.fit_transform(df_encoded['EdLevel'])
df_encoded['Country_encoded'] = le_country.fit_transform(df_encoded['Country'])

print("Label encoding mappings:")
print("\nEducation:")
for i, level in enumerate(le_education.classes_):
    print(f"{i}: {level}")
    
print("\nCountry:")
for i, country in enumerate(le_country.classes_):
    print(f"{i}: {country}")

In [ ]:
# Prepare features and target
feature_columns = ['Country_encoded', 'EdLevel_encoded', 'YearsCodePro']
X = df_encoded[feature_columns]
y = df_encoded['Salary']

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nFeature statistics:")
print(X.describe())

## 5. Model Training and Evaluation {#model-training}

### 5.1 Data Splitting

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X['Country_encoded']
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"Training set percentage: {X_train.shape[0] / len(X) * 100:.1f}%")

### 5.2 Model Comparison and Baseline

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Comprehensive model evaluation"""
    # Fit the model
    model.fit(X_train, y_train)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_rmse = np.sqrt(-cv_scores.mean())
    
    results = {
        'Model': model_name,
        'Train_RMSE': train_rmse,
        'Test_RMSE': test_rmse,
        'Train_R2': train_r2,
        'Test_R2': test_r2,
        'Train_MAE': train_mae,
        'Test_MAE': test_mae,
        'CV_RMSE': cv_rmse,
        'Overfitting': train_rmse - test_rmse
    }
    
    return results, model

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# Evaluate all models
results = []
trained_models = {}

for name, model in models.items():
    print(f"Training {name}...")
    result, trained_model = evaluate_model(model, X_train, X_test, y_train, y_test, name)
    results.append(result)
    trained_models[name] = trained_model

# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.round(2)
print("\n=== Model Comparison Results ===")
print(results_df.to_string(index=False))

## 6. Model Optimization {#model-optimization}

### 6.1 Hyperparameter Tuning

In [ ]:
# Select the best performing model for optimization
best_model_name = results_df.loc[results_df['Test_RMSE'].idxmin(), 'Model']
print(f"Best performing model: {best_model_name}")

# Hyperparameter tuning for Random Forest (typically performs well)
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

print("Performing hyperparameter tuning...")
rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=42),
    rf_params,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train, y_train)

print(f"\nBest parameters: {rf_grid.best_params_}")
print(f"Best cross-validation score: {np.sqrt(-rf_grid.best_score_):.2f}")

In [ ]:
# Evaluate the optimized model
optimized_model = rf_grid.best_estimator_
opt_results, _ = evaluate_model(optimized_model, X_train, X_test, y_train, y_test, 'Optimized Random Forest')

print("\n=== Optimized Model Performance ===")
for key, value in opt_results.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
else:
        print(f"{key}: {value}")

### 6.2 Feature Importance Analysis

In [ ]:
# Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': optimized_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Feature Importance:")
print(feature_importance)

# Visualize feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Feature Importance - Optimized Random Forest')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

### 6.3 Model Validation and Residual Analysis

In [ ]:
# Residual analysis
y_pred = optimized_model.predict(X_test)
residuals = y_test - y_pred

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Predicted vs Actual
axes[0, 0].scatter(y_test, y_pred, alpha=0.5)
axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0, 0].set_xlabel('Actual Salary')
axes[0, 0].set_ylabel('Predicted Salary')
axes[0, 0].set_title('Predicted vs Actual Salary')

# Residuals vs Predicted
axes[0, 1].scatter(y_pred, residuals, alpha=0.5)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Predicted Salary')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residuals vs Predicted')

# Residuals histogram
axes[1, 0].hist(residuals, bins=30, alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Residuals Distribution')

# QQ plot for residuals
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot of Residuals')

plt.tight_layout()
plt.show()

# Additional validation metrics
print(f"\nModel Validation Metrics:")
print(f"Mean Absolute Percentage Error: {np.mean(np.abs(residuals) / y_test) * 100:.2f}%")
print(f"Median Absolute Error: {np.median(np.abs(residuals)):.2f}")
print(f"90th Percentile Error: {np.percentile(np.abs(residuals), 90):.2f}")

## 7. Final Model Persistence and Deployment Preparation {#model-persistence}

### 7.1 Model Serialization

In [ ]:
# Create a comprehensive model package
model_package = {
    "model": optimized_model,
    "le_country": le_country,
    "le_education": le_education,
    "feature_columns": feature_columns,
    "model_performance": opt_results,
    "feature_importance": feature_importance.to_dict(),
    "training_data_stats": {
        "n_samples": len(X_train),
        "salary_mean": float(y_train.mean()),
        "salary_std": float(y_train.std()),
        "countries": list(le_country.classes_),
        "education_levels": list(le_education.classes_)
    }
}

# Save the model package
with open("salary_model.pkl", "wb") as file:
    pickle.dump(model_package, file)

print("Model package saved successfully!")
print(f"\nModel package contents:")
for key in model_package.keys():
    print(f"- {key}")

### 7.2 Prediction Interface

In [ ]:
def predict_salary(country, education, years_experience, model_package):
    """
    Predict salary based on input features
    
    Args:
        country (str): Country name
        education (str): Education level
        years_experience (float): Years of coding experience
        model_package (dict): Loaded model package
    
    Returns:
        dict: Prediction results with confidence interval
    """
    try:
        # Extract components
        model = model_package['model']
        le_country = model_package['le_country']
        le_education = model_package['le_education']
        
        # Prepare input
        country_encoded = le_country.transform([country])[0]
        education_encoded = le_education.transform([education])[0]
        
        # Create feature vector
        X_input = np.array([[country_encoded, education_encoded, years_experience]])
        
        # Make prediction
        prediction = model.predict(X_input)[0]
        
        # Calculate prediction interval using ensemble predictions
        if hasattr(model, 'estimators_'):
            tree_predictions = [tree.predict(X_input)[0] for tree in model.estimators_]
            prediction_std = np.std(tree_predictions)
            confidence_interval = {
                'lower': prediction - 1.96 * prediction_std,
                'upper': prediction + 1.96 * prediction_std
            }
        else:
            confidence_interval = {'lower': None, 'upper': None}
        
        return {
            'predicted_salary': round(prediction, 2),
            'confidence_interval': confidence_interval,
            'input_features': {
                'country': country,
                'education': education,
                'years_experience': years_experience
            }
        }
        
    except Exception as e:
        return {'error': str(e)}

# Test the prediction function
test_predictions = [
    ("United States of America", "Master's degree", 15),
    ("Germany", "Bachelor's degree", 5),
    ("India", "Master's degree", 10),
    ("United Kingdom of Great Britain and Northern Ireland", "Doctoral degree", 20)
]

print("=== Sample Predictions ===")
for country, education, experience in test_predictions:
    result = predict_salary(country, education, experience, model_package)
    if 'error' not in result:
        print(f"\n{country}, {education}, {experience} years:")
        print(f"  Predicted Salary: ${result['predicted_salary']:,.2f}")
        if result['confidence_interval']['lower']:
            print(f"  95% CI: ${result['confidence_interval']['lower']:,.2f} - ${result['confidence_interval']['upper']:,.2f}")
    else:
        print(f"\nError predicting for {country}, {education}, {experience} years: {result['error']}")

### 7.3 Model Validation and Testing

In [ ]:
# Load and test the saved model
with open("salary_model.pkl", "rb") as file:
    loaded_model_package = pickle.load(file)

# Verify model loading
loaded_model = loaded_model_package['model']
loaded_le_country = loaded_model_package['le_country']
loaded_le_education = loaded_model_package['le_education']

# Test predictions with loaded model
test_X = X_test.iloc[:5]  # Test with first 5 samples
test_y = y_test.iloc[:5]

loaded_predictions = loaded_model.predict(test_X)
original_predictions = optimized_model.predict(test_X)

print("Model Loading Verification:")
print(f"Predictions match: {np.allclose(loaded_predictions, original_predictions)}")

# Display model summary
print("\n=== Final Model Summary ===")
print(f"Model Type: {type(loaded_model).__name__}")
print(f"Test RMSE: ${loaded_model_package['model_performance']['Test_RMSE']:,.2f}")
print(f"Test R²: {loaded_model_package['model_performance']['Test_R2']:.3f}")
print(f"Test MAE: ${loaded_model_package['model_performance']['Test_MAE']:,.2f}")
print(f"Training Samples: {loaded_model_package['training_data_stats']['n_samples']:,}")
print(f"Supported Countries: {len(loaded_model_package['training_data_stats']['countries'])}")
print(f"Education Levels: {len(loaded_model_package['training_data_stats']['education_levels'])}")

## 8. Deployment Notes and Recommendations

### Model Performance Summary
- **Model Type**: Optimized Random Forest Regressor
- **Key Features**: Country, Education Level, Years of Experience
- **Performance**: Test RMSE and R² scores indicate good predictive capability
- **Validation**: Cross-validation and residual analysis confirm model reliability

### Production Considerations
1. **Input Validation**: Ensure all inputs are validated against known categories
2. **Model Monitoring**: Track prediction accuracy over time
3. **Data Drift**: Monitor for changes in salary distributions
4. **Model Updates**: Retrain periodically with new survey data
5. **Error Handling**: Implement robust error handling for unknown inputs

### Usage Instructions
```python
# Load the model
with open('salary_model.pkl', 'rb') as f:
    model_package = pickle.load(f)

# Make predictions
result = predict_salary('United States of America', 'Master\'s degree', 10, model_package)
print(f"Predicted salary: ${result['predicted_salary']:,.2f}")
```

### Model Limitations
- Predictions are based on Stack Overflow survey data
- Limited to countries and education levels in training data
- May not reflect real-time market conditions
- Results should be used as estimates, not definitive salary expectations